# Análisis de Ventas de Champán con TBATS

Este cuaderno demuestra el uso del algoritmo TBATS (Trigonometric seasonality, Box-Cox transformation, ARMA errors, Trend and Seasonal components) para el análisis y predicción de series temporales.

**Dataset:** Ventas mensuales de champán (1964-1972)

## 1. Instalación de Dependencias

⚠️ **Nota importante sobre compatibilidad con NumPy:**
La librería `tbats` tiene problemas de compatibilidad con versiones recientes de NumPy.
Instalaremos la libreria `statsforecast` que tambien tiene implementada tbats y funcionan correctamente juntas.

In [ ]:
!pip install statsforecast

## 2. Importar Librerías

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#from tbats import TBATS
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsforecast import StatsForecast
from statsforecast.models import AutoTBATS
import warnings
warnings.filterwarnings('ignore')

print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print("Librerías importadas correctamente")

## 3. Cargar y Explorar los Datos

In [ ]:
# Cargar datos
df = pd.read_csv('Champagne Sales.csv', parse_dates=['Month'], index_col='Month')

# Renombrar columna para facilitar el uso
df.columns = ['Sales']

print("Información del Dataset:")
print(f"Período: {df.index.min()} a {df.index.max()}")
print(f"Total de observaciones: {len(df)}")
print(f"\nPrimeras observaciones:")
print(df.head(10))
print(f"\nEstadísticas descriptivas:")
print(df.describe())

## 4. Visualización Exploratoria

In [ ]:
# Gráfico de la serie temporal
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Serie temporal completa
axes[0].plot(df.index, df['Sales'], linewidth=2, color='darkblue')
axes[0].set_title('Ventas Mensuales de Champán (1964-1972)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Fecha')
axes[0].set_ylabel('Ventas')
axes[0].grid(True, alpha=0.3)

# Boxplot por año para ver estacionalidad
df_year = df.copy()
df_year['Year'] = df_year.index.year
df_year['Month_num'] = df_year.index.month
df_pivot = df_year.pivot_table(values='Sales', index='Month_num', columns='Year')
axes[1].boxplot([df_pivot.loc[i].dropna() for i in range(1, 13)],
                labels=['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun',
                       'Jul', 'Ago', 'Sep', 'Oct', 'Nov', 'Dic'])
axes[1].set_title('Distribución de Ventas por Mes', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Mes')
axes[1].set_ylabel('Ventas')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 5. División de Datos: Entrenamiento y Prueba

In [ ]:
# Dividir en entrenamiento (80%) y prueba (20%)
train_size = int(len(df) * 0.8)
train_data = df.iloc[:train_size]['Sales'].values
test_data = df.iloc[train_size:]['Sales'].values
test_index = df.iloc[train_size:].index


print(f"Entrenamiento: {train_size} observaciones ({df.index[0]} a {df.index[train_size-1]})")
print(f"Prueba: {len(test_data)} observaciones ({test_index[0]} a {test_index[-1]})")

# Visualizar la división
plt.figure(figsize=(14, 6))
plt.plot(df.index[:train_size], train_data, label='Entrenamiento', linewidth=2, color='blue')
plt.plot(test_index, test_data, label='Prueba', linewidth=2, color='orange')
plt.axvline(x=df.index[train_size-1], color='red', linestyle='--', linewidth=2, label='División')
plt.title('División de Datos: Entrenamiento vs Prueba', fontsize=14, fontweight='bold')
plt.xlabel('Fecha')
plt.ylabel('Ventas')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 6. Entrenamiento del Modelo TBATS

TBATS es especialmente útil para series temporales con:
- Múltiples patrones estacionales
- Estacionalidad compleja
- Datos con tendencia

**Parámetros principales:**
- `season_length`: Períodos estacionales (12 para datos mensuales)

In [ ]:
from statsforecast import StatsForecast
from statsforecast.models import AutoTBATS

In [ ]:
estimator = StatsForecast(
    models=[AutoTBATS(season_length=12)],
    freq='MS'
)

# Convert train_data to the required DataFrame format for statsforecast
train_df = pd.DataFrame({
    'unique_id': 'champagne_sales',
    'ds': df.index[:train_size],
    'y': train_data
})

# Ajustar el modelo
fitted_model = estimator.fit(train_df)

print("Modelo entrenado\n")

In [ ]:
fitted_model.forecast_fitted_values

In [ ]:
print("📋 Resumen del modelo:")
# Accessing the summary might be different depending on the statsforecast version
# Let's try to print the fitted model object, it might provide some info
print(fitted_model)

# If a summary method exists, uncomment the line below:
# print(fitted_model.summary())

## 7. Realizar Predicciones

In [ ]:
# Predicciones para el conjunto de prueba
n_forecast = len(test_data)

# Use the fitted_model to predict on the test set
# The predict method in statsforecast takes the number of steps as the first argument
test_pred_df = fitted_model.predict(n_forecast)

# Extract the prediction array and confidence intervals from the prediction DataFrame
y_pred = test_pred_df['AutoTBATS'].values

# Assuming the statsforecast output structure has columns like 'AutoTBATS-lo-95' and 'AutoTBATS-hi-95'
conf_int_cols = [col for col in test_pred_df.columns if '-lo-' in col or '-hi-' in col]
if len(conf_int_cols) >= 2:
    conf_int = {
        'lower_bound': test_pred_df[conf_int_cols[0]].values,
        'upper_bound': test_pred_df[conf_int_cols[1]].values
    }
else:
    # Handle case where confidence intervals are not available
    conf_int = {'lower_bound': np.nan, 'upper_bound': np.nan}
    print("Warning: Confidence intervals not found in the prediction output.")


print(f"✅ {n_forecast} predicciones generadas")
print(f"\nPrimeras 5 predicciones:")
pred_df = pd.DataFrame({
    'Fecha': test_index[:5],
    'Real': test_data[:5],
    'Predicción': y_pred[:5],
    'Error': test_data[:5] - y_pred[:5]
})
print(pred_df.to_string(index=False))

In [ ]:
test_data

## 8. Evaluación del Modelo

In [ ]:
# Calcular métricas de error
mse = mean_squared_error(test_data, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(test_data, y_pred)
mape = np.mean(np.abs((test_data - y_pred) / test_data)) * 100

print(f"MSE (Error Cuadrático Medio):     {mse:,.2f}")
print(f"RMSE (Raíz del MSE):              {rmse:,.2f}")
print(f"MAE (Error Absoluto Medio):       {mae:,.2f}")
print(f"MAPE (Error Porcentual Absoluto): {mape:.2f}%")


# Interpretación
print(f"\n💡 Interpretación:")
print(f"En promedio, las predicciones difieren de los valores reales en {mae:,.0f} unidades.")
print(f"El error porcentual promedio es de {mape:.1f}%.")

## 9. Visualización de Resultados

In [ ]:
# Gráfico principal: Comparación de predicciones
fig, axes = plt.subplots(2, 1, figsize=(14, 12))

# 1. Serie temporal completa con predicciones
axes[0].plot(df.index[:train_size], train_data, label='Entrenamiento', linewidth=2, color='blue')
axes[0].plot(test_index, test_data, label='Real (Prueba)', linewidth=2, color='green' )
axes[0].plot(test_index, y_pred, label='Predicción TBATS', linewidth=2, color='red')
axes[0].axvline(x=df.index[train_size-1], color='black', linestyle=':', linewidth=1.5, alpha=0.5)
axes[0].set_title('TBATS: Predicciones vs Valores Reales', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Fecha')
axes[0].set_ylabel('Ventas')
axes[0].legend(loc='upper left')
axes[0].grid(True, alpha=0.3)

# 2. Zoom en el período de prueba
axes[1].plot(test_index, test_data, label='Real', linewidth=2.5, color='green')
axes[1].plot(test_index, y_pred, label='Predicción', linewidth=2.5, color='red')
axes[1].set_title('Detalle del Período de Prueba', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Fecha')
axes[1].set_ylabel('Ventas')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 11. Predicciones Futuras

Vamos a generar predicciones para los próximos 12 meses más allá de nuestros datos.

In [ ]:
# Reentrenar con todos los datos para predicciones futuras
print("Reentrenando modelo con todos los datos...\n")

# Convert the full dataframe to the required format for statsforecast
full_df = pd.DataFrame({
    'unique_id': 'champagne_sales',
    'ds': df.index,
    'y': df['Sales'].values
})

full_model = estimator.fit(full_df)

# Predecir los próximos 12 meses
future_steps = 12
# The predict method in statsforecast takes the number of steps as the first argument
future_pred_df = full_model.predict(future_steps)

# The prediction is in a DataFrame, extract the prediction array
future_pred = future_pred_df['AutoTBATS'].values

# Create future dates index
last_date = df.index[-1]
future_dates = pd.date_range(start=last_date + pd.DateOffset(months=1), periods=future_steps, freq='MS')


# Visualizar predicciones futuras
plt.figure(figsize=(14, 6))
plt.plot(df.index, df['Sales'], label='Datos Históricos', linewidth=2, color='blue')
plt.plot(future_dates, future_pred, label='Predicción Futura', linewidth=2.5,
        color='red', linestyle='--', marker='o', markersize=6)
plt.axvline(x=last_date, color='black', linestyle=':', linewidth=2, label='Última observación')
plt.title('Predicción de Ventas para los Próximos 12 Meses', fontsize=14, fontweight='bold')
plt.xlabel('Fecha')
plt.ylabel('Ventas')
plt.legend()
plt.show()

# Tabla de predicciones futuras
future_df = pd.DataFrame({
    'Fecha': future_dates,
    'Predicción': future_pred,
})


In [ ]:
print("\nPREDICCIONES PARA LOS PRÓXIMOS 12 MESES")
print(future_df.to_string(index=False))


## 13. Conclusiones

### Ventajas de TBATS:
- ✅ Maneja múltiples estacionalidades automáticamente
- ✅ Incluye transformación Box-Cox para estabilizar varianza
- ✅ Modela tendencias y patrones complejos
- ✅ Proporciona intervalos de confianza

### Desventajas:
- ⚠️ Tiempo de entrenamiento puede ser largo
- ⚠️ Requiere versiones específicas de librerías (problemas con NumPy)
- ⚠️ Menos interpretable que modelos más simples

### Recomendaciones:
- Para series con patrones estacionales claros, TBATS es muy efectivo
- Considerar modelos más simples (SARIMA) si el tiempo de cómputo es importante
- Probar diferentes configuraciones para optimizar resultados

## 14. Exportar Resultados

In [ ]:
# Crear DataFrame con all the results
results_export = pd.DataFrame({
    'Fecha': test_index,
    'Valor_Real': test_data,
    'Prediccion_TBATS': y_pred,
    'Error': test_data - y_pred,
    'Error_Porcentual': ((test_data - y_pred) / test_data) * 100,
})

# Guardar resultados
results_export.to_csv('tbats_resultados.csv', index=False)
future_df.to_csv('tbats_predicciones_futuras.csv', index=False)

print("Resultados exportados:")
print("   - tbats_resultados.csv (comparación con datos de prueba)")
print("   - tbats_predicciones_futuras.csv (predicciones para 12 meses)")

